# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 59.4 MB/s eta 0:00:0

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('openai')

In [12]:
# If you're using colab, run this
os.environ['OPENAI_API_KEY'] = "openai"

In [13]:
llm = OpenAI(temperature=0)

In [15]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [16]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [17]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [18]:
from langchain_community.document_loaders import WebBaseLoader

In [19]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [20]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [21]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [29]:
tools = [
    Tool(
      name="Sonnets QA System",
      func=state_of_union.run,
      description="Useful for answering questions about Shakespeare's Sonnets. Input should be a fully formed question.",
  ),
    Tool(
    name="Ruff QA System",
    func=ruff.run,
    description="Useful for answering questions about Ruff, the Python linter. Input should be a fully formed question.",
),
]

In [30]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [32]:
agent.invoke("What does Sonnet 18 say about beauty?")




> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What does Sonnet 18 say about beauty?"
Observation:  Sonnet 18 suggests that beauty is timeless and that even in the future, people will still admire and praise it. It also implies that beauty is a gift from nature and that it should be celebrated and appreciated.
Thought: I now know the final answer.
Final Answer: Sonnet 18 says that beauty is timeless and should be celebrated and appreciated.

> Finished chain.


{'input': 'What does Sonnet 18 say about beauty?',
 'output': 'Sonnet 18 says that beauty is timeless and should be celebrated and appreciated.'}

In [33]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both Python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and type checkers like Mypy. This means that you can use Ruff alongside these tools to get more comprehensive feedback on your code.

3. Automatic fixing of lint violations: Unlike Flake8, Ruff is capable of automatically fixing its own lint violations. This can save you time and effort when fixing issues in your code.

4. Native implementation of popular Flake8 plugins: Ruff re-imple

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff offers a larger rule set, better compatibility with other tools, automatic fixing of lint violations, and native implementation of popular Flake8 plugins, making it a more comprehensive and user-friendly choice compared to Flake8.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [34]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=state_of_union.run,
        description="Useful for answering questions about Shakespeare's Sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for answering questions about Ruff, the Python linter. Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [35]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [36]:
agent.invoke("What does Sonnet 18 say about beauty?")




> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What does Sonnet 18 say about beauty?"
Observation:  Sonnet 18 suggests that beauty is timeless and that even in the future, people will still admire and praise it. It also implies that beauty is a gift from nature and that it should be celebrated and appreciated.


> Finished chain.


{'input': 'What does Sonnet 18 say about beauty?',
 'output': ' Sonnet 18 suggests that beauty is timeless and that even in the future, people will still admire and praise it. It also implies that beauty is a gift from nature and that it should be celebrated and appreciated.'}

In [37]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both Python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.

3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.

4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.

5.

{'input': 'Why use ruff over flake8?',
 'output': '\nThere are a few reasons why someone might choose to use Ruff over Flake8:\n\n1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.\n\n2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.\n\n3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.\n\n4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.\n\n5. Can be used as a formatter: Ruff can be used as a formatter, making it a more versatile tool than Flake8.\n\nUltimately, the choice between Ruff and Flake8 will depend on the specific needs and preferences

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [38]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=state_of_union.run,
        description="Useful for answering questions about Shakespeare's Sonnets. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for answering questions about Ruff, the Python linter. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [39]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [40]:
agent.invoke(
    "Which sonnet discusses the passing of time and decay? Does Ruff have a tool that could help automate code formatting to save time?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to find the answer to the first question, and the Ruff QA System to find the answer to the second question.
Action: Sonnets QA System
Action Input: "Which sonnet discusses the passing of time and decay?"
Observation:  Sonnet XIII, LXV, and LXI all discuss the passing of time and decay.
Thought: I should narrow down the search to find the specific sonnet.
Action: Sonnets QA System
Action Input: "Which sonnet discusses the passing of time and decay and mentions the word 'decay'?"
Observation:  Sonnet XIII.
Thought: I should now switch to the Ruff QA System to find the answer to the second question.
Action: Ruff QA System
Action Input: "Does Ruff have a tool that could help automate code formatting to save time?"
Observation:  Yes, Ruff has a formatter that can be used to automatically format code. It is designed to be a drop-in replacement for Black and aims to produce near-identical output when run over Black-fo

{'input': 'Which sonnet discusses the passing of time and decay? Does Ruff have a tool that could help automate code formatting to save time?',
 'output': 'Sonnet XIII discusses the passing of time and decay, and Ruff has a tool that can help automate code formatting to save time.'}